# Deep Learning for Forecasting using GluonTS

## 1. Intro

### What is GluonTS

GluonTS is an open source Python library for time series modeling, with a focus on deep learning architectures for forecasting.
It provides several tools to support development and experimentation with new models, as well as pre-implemented models from the literature.
We originally developed GluonTS on top of MXNet's Gluon API (hence the name), but we're making all parts of the library compatible with PyTorch models too.

Target users:
* **Researchers** who want to try out novel architectures for forecasting and compare them to the state of the art
* **Data scientist/solution architects** who want to experiment with several solution to solve business problems
* **Machine learning engineer** who need to integrate forecasting models in production pipelines

### What you're going to learn

* Handle datasets in GluonTS (transform them and iterate them)
* Forecast and do backtest evaluations given a model ("predictors")
* Implement a neural-network-based architecture using PyTorch, and train it over a dataset
* Use pre-implemented architectures ("estimators")

## 2. Overview

In [ ]:
from typing import List, Callable, Dict
from itertools import islice
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
from gluonts.dataset.repository.datasets import get_dataset

For this tutorial, we're going to use the hourly dataset from the M4 competition:

In [ ]:
dataset = get_dataset("m4_hourly")

In [ ]:
len(dataset.train)

In [ ]:
pprint(next(iter(dataset.train)))

In [ ]:
def to_series(entry: Dict) -> pd.Series:
    index = pd.date_range(start=entry["start"], periods=len(entry["target"]), freq=entry["start"].freq)
    return pd.Series(entry["target"], index=index)

In [ ]:
plt.figure(figsize=(20, 15))

for idx, entry in enumerate(islice(dataset.train, 9)):
    ax = plt.subplot(3, 3, idx+1)
    to_series(entry).plot()
    plt.grid()

plt.gcf().tight_layout()
plt.show()

In [ ]:
from gluonts.model.seasonal_naive import SeasonalNaivePredictor

In [ ]:
predictor_naive = SeasonalNaivePredictor(
    freq=dataset.metadata.freq,
    prediction_length=dataset.metadata.prediction_length,
    season_length=7 * 24,
)

In [ ]:
forecasts = list(predictor_naive.predict(dataset.train))

In [ ]:
forecasts[0]

In [ ]:
plt.figure(figsize=(20, 15))

for idx, (entry, f) in islice(enumerate(zip(dataset.train, forecasts)), 9):
    ax = plt.subplot(3, 3, idx+1)
    to_series(entry)[-3*24:].plot()
    f.plot(color="r")
    plt.grid()
    plt.title(f"item_id: {f.item_id}")

plt.gcf().tight_layout()
plt.show()

In [ ]:
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import backtest_metrics

In [ ]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

In [ ]:
aggregate_metrics_naive, entrywise_metrics_naive = backtest_metrics(dataset.test, predictor_naive, evaluator=evaluator)

In [ ]:
pd.DataFrame.from_records(
    [aggregate_metrics_naive],
    index=["Naive seasonal"]
).transpose()

In [ ]:
entrywise_metrics_naive

## 3. Implementing a PyTorch model: a probabilistic feed-forward network

In [ ]:
import torch
import torch.nn as nn

We will use a pretty simple model, based on a feed-forward network whose output layer produces the parameters of a Student's t-distribution at each time step in the prediction range.

<img src="figures/feedforward.png" style="width: 300px; margin-left: auto; margin-right: auto;">

We will define two networks based on this idea:
* The `TrainingFeedForwardNetwork` computes the loss associated with given observations, i.e. the negative log-likelihood of the observations according to the output distribution; this will be used during training.
* The `SamplingFeedForwardNetwork` will be used at inference time: this uses the output distribution to draw a sample of a given size, as a way to encode the predicted distribution.

In [ ]:
def mean_abs_scaling(context, min_scale=1e-5):
    return context.abs().mean(1).clamp(min_scale, None).unsqueeze(1)

In [ ]:
def no_scaling(context):
    return torch.ones(context.shape[0], 1)

In [ ]:
class TrainingFeedForwardNetwork(nn.Module):
    distr_type = torch.distributions.StudentT
    
    def __init__(
        self,
        prediction_length: int,
        context_length: int,
        hidden_dimensions: List[int],
        batch_norm: bool=False,
        scaling: Callable=mean_abs_scaling,
    ) -> None:
        super().__init__()
        
        assert prediction_length > 0
        assert context_length > 0
        assert len(hidden_dimensions) > 0
        
        self.prediction_length = prediction_length
        self.context_length = context_length
        self.hidden_dimensions = hidden_dimensions
        self.batch_norm = batch_norm
        self.scaling = scaling
        
        dimensions = [context_length] + hidden_dimensions[:-1]

        modules = []
        for in_size, out_size in zip(dimensions[:-1], dimensions[1:]):
            modules += [self._linear_layer(in_size, out_size), nn.ReLU()]
            if batch_norm:
                modules.append(nn.BatchNorm1d(units))
        modules.append(self._linear_layer(dimensions[-1], prediction_length * hidden_dimensions[-1]))
        self.nn = nn.Sequential(*modules)
        
        self.df_proj = nn.Sequential(self._linear_layer(hidden_dimensions[-1], 1), nn.Softplus())
        self.loc_proj = self._linear_layer(hidden_dimensions[-1], 1)
        self.scale_proj = nn.Sequential(self._linear_layer(hidden_dimensions[-1], 1), nn.Softplus())
    
    @staticmethod
    def _linear_layer(dim_in, dim_out):
        lin = nn.Linear(dim_in, dim_out)
        torch.nn.init.uniform_(lin.weight, -0.07, 0.07)
        torch.nn.init.zeros_(lin.bias)
        return lin
    
    def distr_and_scale(self, context):
        scale = self.scaling(context)
        scaled_context = context / scale
        nn_out = self.nn(scaled_context)
        nn_out_reshaped = nn_out.reshape(-1, self.prediction_length, self.hidden_dimensions[-1])
        
        distr_args = (
            2.0 + self.df_proj(nn_out_reshaped).squeeze(dim=-1),
            self.loc_proj(nn_out_reshaped).squeeze(dim=-1),
            self.scale_proj(nn_out_reshaped).squeeze(dim=-1),
        )
        distr = net.distr_type(*distr_args)
        
        return distr, scale
    
    def forward(self, context, target):
        assert context.shape[-1] == self.context_length
        assert target.shape[-1] == self.prediction_length
        
        distr, scale = self.distr_and_scale(context)
        loss = (-distr.log_prob(target / scale) + torch.log(scale)).mean(dim=1)
        
        return loss

Shape of the inputs:
* `context` has size `batch_size x context_length`
* `target` has size `batch_size x prediction_length`

Shape of the output:
* `loss` has size `batch_size`

In [ ]:
class SamplingFeedForwardNetwork(TrainingFeedForwardNetwork):
    def __init__(self, *args, num_samples: int = 1000, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_samples = num_samples
        
    def forward(self, context):
        assert context.shape[-1] == self.context_length
        
        distr, scale = self.distr_and_scale(context)
        sample_perm = distr.sample((self.num_samples, )) * scale
        sample = sample_perm.permute(1, 0, 2)
        
        return sample

Shape of the inputs:
* `context` has size `batch_size x context_length`

Shape of the output:
* `sample` has size `batch_size x num_samples x prediction_length`

We can now instantiate the training network, and explore its set of parameters.

In [ ]:
context_length = 2 * 7 * 24
prediction_length = dataset.metadata.prediction_length
hidden_dimensions = [96, 48]

In [ ]:
net = TrainingFeedForwardNetwork(
    prediction_length=prediction_length,
    context_length=context_length,
    hidden_dimensions=hidden_dimensions,
    batch_norm=False,
    scaling=mean_abs_scaling,
)

In [ ]:
sum(np.prod(p.shape) for p in net.parameters())

In [ ]:
for p in net.parameters():
    print(p.shape)

### Defining the training data loader

We now set up the data loader which will yield batches of data to train on. Starting from the original dataset, the data loader is configured to apply the following transformation, which does essentially two things:
* Replaces `nan`s in the target field with a dummy value (zero), and adds a field indicating which values were actually observed vs imputed this way.
* Slices out training instances of a fixed length randomly from the given dataset; these will be stacked into batches by the data loader itself.

In [ ]:
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.loader import TrainDataLoader
from gluonts.torch.batchify import batchify
from gluonts.torch.support.util import copy_parameters
from gluonts.torch.model.predictor import PyTorchPredictor
from gluonts.transform import Chain, AddObservedValuesIndicator, InstanceSplitter, ExpectedNumInstanceSampler

In [ ]:
transformation = Chain([
    AddObservedValuesIndicator(
        target_field=FieldName.TARGET,
        output_field=FieldName.OBSERVED_VALUES,
    ),
    InstanceSplitter(
        target_field=FieldName.TARGET,
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        train_sampler=ExpectedNumInstanceSampler(num_instances=1),
        past_length=context_length,
        future_length=prediction_length,
        time_series_fields=[FieldName.OBSERVED_VALUES],
    ),
])

<img src="figures/training_data_iter.png" style="width: 800px; margin-left: auto; margin-right: auto;">

In [ ]:
batch_size = 32
num_batches_per_epoch = 100

In [ ]:
data_loader = TrainDataLoader(
    dataset.train,
    transform=transformation,
    batch_size=batch_size,
    stack_fn=batchify,
    num_batches_per_epoch=num_batches_per_epoch
)

In [ ]:
batch = next(iter(data_loader))

In [ ]:
batch["past_target"]

In [ ]:
batch["past_target"].shape

In [ ]:
batch["past_observed_values"]

In [ ]:
batch["future_target"]

In [ ]:
batch["future_target"].shape

In [ ]:
batch.keys()

### Train the model

We can now train the model using any of the available optimizers from PyTorch:

In [ ]:
optimizer = torch.optim.Adam(net.parameters())

for epoch_no in range(20):
    sum_epoch_loss = 0.0
    for batch_no, batch in enumerate(data_loader, start=1):
        optimizer.zero_grad()

        loss_vec = net(context=batch["past_target"], target=batch["future_target"])
        loss = loss_vec.mean()
        loss.backward()
        
        optimizer.step()
        
        sum_epoch_loss += loss.detach().numpy().item()
        
    print(f"{epoch_no}: {sum_epoch_loss / num_batches_per_epoch}")

### Create predictor out of the trained model, and test it

We now have a trained model, whose parameters can be copied over to a `SamplingFeedForwardNetwork` object: we will wrap this into a `PyTorchPredictor` that can be used for inference tasks.

In [ ]:
pred_net = SamplingFeedForwardNetwork(
    prediction_length=net.prediction_length,
    context_length=net.context_length,
    hidden_dimensions=net.hidden_dimensions,
    batch_norm=net.batch_norm,
)
copy_parameters(net, pred_net)

feedforward = PyTorchPredictor(
    prediction_length=prediction_length, freq = dataset.metadata.freq, 
    input_names = ["past_target"], prediction_net=pred_net, batch_size=32, input_transform=transformation,
    device=None
)

For example, we can do backtesting on the test dataset: in what follows, `make_evaluation_predictions` will slice out the trailing `prediction_length` observations from the test time series, and use the given predictor to obtain forecasts for the same time range.

<img src="figures/prediction_data_iter.png" style="width: 800px; margin-left: auto; margin-right: auto;">

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,
    predictor=feedforward,
    num_samples=1000,
)

forecasts_feedforward = list(forecast_it)
tss_feedforward = list(ts_it)

Once we have the forecasts, we can plot them:

In [ ]:
plt.figure(figsize=(20, 15))
date_formatter = matplotlib.dates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts_feedforward, tss_feedforward)), 9):
    ax = plt.subplot(3, 3, idx+1)
    plt.plot(ts[-5 * prediction_length:], label="target")
    forecast.plot(color="g")
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formatter)
    
plt.gcf().tight_layout()
plt.legend()
plt.show()

And we can compute evaluation metrics, that summarize the performance of the model on our test data.

In [ ]:
aggregate_metrics_feedforward, entrywise_metrics_feedforward = evaluator(
    iter(tss_feedforward), iter(forecasts_feedforward), num_series=len(dataset.test)
)

In [ ]:
pd.DataFrame.from_records(
    [aggregate_metrics_naive, aggregate_metrics_feedforward],
    index=["Naive seasonal", "Feed-forward"]
).transpose()

## 4. Using pre-implemented models

In [ ]:
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.mx.trainer import Trainer

MQ-CNN (Wen et al., 2017) is a *sequence-to-sequence* model, using
* a convolutional network as *encoder*
* a feed-forward network as *decoder*

<img src="figures/mq_rnn.png" style="width: 600px; margin-left: auto; margin-right: auto;">

The output of the decoder consists of pre-defined *quantiles* of the predicted distribution, optimized using the *quantile loss* (also known as *pinball loss*):

$$ L_\alpha(\hat z, z) = \begin{cases} \alpha (\hat z - z) & \mbox{if}\ \hat z \geq z \\ (\alpha - 1) (\hat z - z) & \mbox{otherwise} \end{cases}$$

In [ ]:
mqcnn_estimator = MQCNNEstimator(
    freq=dataset.metadata.freq,
    prediction_length=dataset.metadata.prediction_length,
    quantiles=[0.1, 0.5, 0.9],
    trainer=Trainer(epochs=20),
)

In [ ]:
mqcnn = mqcnn_estimator.train(dataset.train)

In [ ]:
mqcnn

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,
    predictor=mqcnn,
    num_samples=100,
)

forecasts_mqcnn = list(forecast_it)
tss_mqcnn = list(ts_it)

In [ ]:
plt.figure(figsize=(20, 15))
date_formatter = matplotlib.dates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts_mqcnn, tss_mqcnn)), 9):
    ax = plt.subplot(3, 3, idx+1)
    
    plt.plot(ts[-5 * prediction_length:], label="target")
    forecast.plot(color="g")
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formatter)
    
plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
aggregate_metrics_mqcnn, entrywise_metrics_mqcnn = evaluator(
    iter(tss_mqcnn), iter(forecasts_mqcnn), num_series=len(dataset.test)
)

In [ ]:
pd.DataFrame.from_records(
    [aggregate_metrics_naive, aggregate_metrics_feedforward, aggregate_metrics_mqcnn],
    index=["Naive seasonal", "Feed-forward", "MQ-CNN"]
).transpose()

## 5. Additional features

Several additional features in GluonTS we did not cover here:

* Models:
    * Wavenet [van den Oord et al., 2016]
    * Deep State Space Models [Rangapuram et al., 2018]
    * Deep Factors [Wang et al. 2019]
    * DeepAR [Flunkert et al., 2020]
    * Attention-based models (transformers)
    * Gaussian processes, Temporal point processes, ...
* Training options:
    * Monitoring validation loss
    * Early stopping
    * Model averaging
* Serialization/deserialization of models
* Helpers to train & deploy models in the cloud, using Amazon SageMaker